# Monte Carlo Simulation - Client for Predator-Prey Model

## Overview

The Uncertainty Quantification (UQ) problem in this context is to evaluate the uncertainty in the predator-prey model outcomes due to **uncertainty in the initial conditions or parameters**, specifically we want to estimate the expected value of the prey population $\mathbb{E}[u_1(T)]$ at a future time T given some uncertainty in the initial conditions.

As explained in the file [predprey_UQ.ipynb](https://github.com/MathSEE-Modeling-Week/Modeling-Week/blob/main/Models/Predator-Prey/predprey_UQ.ipynb), the uncertainty in the predator-prey model comes from the initial conditions. The true initial condition $\bar{u}_0 = [0.5, 2]$ is perturbed with a uniform distribution $\mathcal{U}(\Gamma)$ over a square region $\Gamma$ of radius $\delta = 0.2$. This perturbed initial condition is then used to run a Monte Carlo simulation.

The key objective is to calculate an estimate of $\mathbb{E}[u_1(T)]$, the expected prey population at time $T = 6$, using the **Monte Carlo (MC) method**.

## Step-by-Step Process

### 1. Sampling Initial Conditions

The third file defines that the uncertain initial condition follows a uniform distribution $\mathcal{U}(\Gamma)$, where $\Gamma$ is a square region centered at $\bar{u}_0 = [0.5, 2]$ with side lengths $2\delta = 0.4$. We generate $N = 1000$ Monte Carlo samples of initial conditions within this region.

### 2. Solving the Lotka-Volterra Equations

For each sampled initial condition, we solve the **Lotka-Volterra equations** over the time interval $[0, T]$, where $T = 6$. The equations describe the dynamics between prey and predator populations, with fixed parameter values $\theta_1 = \theta_2 = \theta_{12} = \theta_{21} = 1$, as set in [predprey_UQ.ipynb](https://github.com/MathSEE-Modeling-Week/Modeling-Week/blob/main/Models/Predator-Prey/predprey_UQ.ipynb).

We use the **umbridge model** to solve the system of ODEs for each initial condition.

### 3. Monte Carlo Estimator for Expected Prey Population

Once the prey population $u_1(T)$ is computed for all $N = 1000$ samples, we estimate the expected value $\mathbb{E}[u_1(T)]$ by averaging the results:

\begin{equation}
\mathbb{E}[u_1(T)] \approx \frac{1}{N} \sum_{i=1}^N u_1^{(i)}(T)
\end{equation}

### 4. Comparison with Unperturbed Initial Condition

We also compute the prey population $u_1(T)$ for the **unperturbed initial condition** $\bar{u}_0 = [0.5, 2]$ to provide a baseline for comparison with the perturbed trajectories.

### 5. Visualization of Results

The results are visualized using a histogram of the Monte Carlo samples of $u_1(T)$. The mean (MC estimator) and the prey population for the unperturbed trajectory are marked on the plot for comparison.


## Conclusion

This file demonstrates the use of the Monte Carlo method for **Uncertainty Quantification** in the Predator-Prey model. By generating multiple samples of the uncertain initial conditions and solving the Lotka-Volterra equations for each, we obtain an estimate of the expected prey population at a future time T. The results are compared to the unperturbed scenario to highlight the effects of uncertainty on the system dynamics.

### Task:
Try implementing a MC simulation using the umbridge framework, where the predator-prey model is running as a server see [here](https://github.com/MathSEE-Modeling-Week/Modeling-Week/blob/main/Models/Predator-Prey/predprey_server.ipynb) and the MC simulation as an umbridge client.

**In the next section you cand find an example code.**



## Client for MC simulation
In the following you can see the client code corresponding to the MC simulation. Be sure that your model is running [predprey_server.ipynb](https://github.com/MathSEE-Modeling-Week/Modeling-Week/blob/main/Models/Predator-Prey/predprey_server.ipynb).

In [3]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import umbridge

# Connect to the umbridge model
model = umbridge.HTTPModel("http://0.0.0.0:4242", "forward")

# Define parameters
theta1, theta2, theta12, theta21 = 1.0, 1.0, 1.0, 1.0
T = 6
u0_bar = np.array([0.5, 2])  # Example unperturbed initial condition

# Generate perturbed initial conditions
delta = 0.2
N = 1000  # Number of MC samples

np.random.seed(42)

initial_conditions = u0_bar + delta * (2 * np.random.rand(N, 2) - 1)

# Solve ODE for each initial condition using the umbridge model
u1_T_values = []  # List for storing values of u1(T), i.e., prey, at T=6
for u0 in initial_conditions:
    # Create the input for the model
    parameters = [T, u0[0], u0[1], theta1, theta2, theta12, theta21]
    result = model([parameters])
    u1_T_values.append(result[0][0])  # Extract u1(T) from the result

# Calculate MC estimator
u1_T_mean = np.mean(u1_T_values)
print(f"MC estimator for expected value of u1(T) at T={T}: {u1_T_mean}")

# Solve ODE for the unperturbed initial condition using the umbridge model
parameters_unperturbed = [T, u0_bar[0], u0_bar[1], theta1, theta2, theta12, theta21]
result_unperturbed = model([parameters_unperturbed])
u1_T_unperturbed = result_unperturbed[0][0]
print(f"Value of u1(T) for the unperturbed trajectory for T=6: {u1_T_unperturbed}")

MC estimator for expected value of u1(T) at T=6: 1.5181587659330416
Value of u1(T) for the unperturbed trajectory for T=6: 1.4098300518932472
